In [1]:
!pip install transformers[torch] accelerate
!pip install datasets
!pip install sentence_transformers
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import pandas as pd
import numpy as np
from numba import cuda
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm import tqdm
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df1=pd.read_csv(r'/content/drive/MyDrive/Trailadata_preprocessed.csv')
df1.head()

,File Name,Transcript,Label,tokenized_Transcript,clean_text
0,trial_truth_002.txt,"All of us, who have represented people for yea...",truthful,all of us who have represented people for year...,represent people year system letter prisioner ...
1,trial_truth_001.txt,Reforming the criminal justice system matters ...,truthful,reforming the criminal justice system matters ...,reform criminal justice system matter law legi...
2,trial_truth_006.txt,They really didn't tell me anything. They said...,truthful,they really did n't tell me anything they said...,tell say accident say accident ian fine accide...
3,trial_truth_004.txt,"I do. I was, uh... in the office, and I got a ...",truthful,i do i was uh in the office and i got a call u...,uh office get uh maybe close o clock remember ...
4,trial_truth_003.txt,"But yes, I was there. Yep, I stayed. Uh ... Ye...",truthful,but yes i was there yep i stayed uh yep prob y...,yes yep stay uh yep prob yes yes


In [5]:
df1.drop(columns=['File Name','Transcript','tokenized_Transcript'],inplace=True)

In [6]:
import spacy
from collections import Counter

!python -m spacy download en_core_web_lg

# Load Spacy's English language model
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
# Define a function to tokenize text
def tokenize(txt):
    # Remove punctuation and double space, and convert to lowercase:
    tokens = [token.text.lower() for token in txt if not (token.is_punct or token.is_space)]
    # Join tokens back into a string
    tokens = " ".join(tokens)
    return tokens

# Apply the preprocessing pipeline using nlp.pipe
tokenized_txt = []

for txt in nlp.pipe(df1['clean_text']):
    tokenized_txt.append(tokenize(txt))
# Add the preprocessed text as a new column in the dataframe
df1['tokenized_Transcript'] = tokenized_txt

df1.drop(columns=['clean_text'],inplace=True)
df1['Label'] = df1['Label'].map({'truthful': 1, 'decpetive': 0})
df1['Label'].fillna(0, inplace=True)
df1['Label'] = df1['Label'].astype(int)


In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Preprocess function to tokenize input text and format labels
def preprocess_function(examples):
    inputs = examples["tokenized_Transcript"]
    targets = examples["Label"]

    # Convert labels to lists
    targets = [[label] for label in targets]

    # Ensure that inputs are lists of strings
    if isinstance(inputs, str):
        inputs = [inputs]

    # Tokenize inputs
    inputs_tokenized = tokenizer(inputs, padding=True, truncation=True)['input_ids']

    return {'input_ids': inputs_tokenized, 'labels': targets}


In [10]:
def create_dataset(df1, seed=42, cv=10):
    train = {f'split_{i+1}': [] for i in range(cv)}
    test = {f'split_{i+1}': [] for i in range(cv)}

    np.random.seed(seed)

    for k in train.keys():
        train[k] = df1.sample(frac=0.8, random_state=seed)
        test[k] = df1.drop(train[k].index)

    return train, test


In [12]:
seed = 42
numcv = 10
num_epochs = 3
model_size = 'base'
#!pip install --upgrade transformers accelerate
import accelerate
from sklearn.metrics import accuracy_score
train, test = create_dataset(df1, seed=seed, cv=numcv)
results = {}
ground_truth = {}

In [13]:
# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForSeq2SeqLM
for sp in tqdm(train.keys()):
    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
    # Tokenize and preprocess data
    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",  # The output directory
        logging_dir="./logs",
        evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch
        learning_rate=5e-4,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    # Define trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_test,
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()
    # Get predictions
    predictions = trainer.predict(data_test)
    pred_labels = np.argmax(predictions.predictions, axis=1)

    # Store predictions in the original DataFrame
    df1.loc[test[sp].index, 'Prediction'] = pred_labels

    # Clean up
    del model
    torch.cuda.empty_cache()


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.288958
2,No log,2.755078
3,No log,2.441389


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


100%|██████████| 10/10 [05:49<00:00, 34.91s/it]


In [14]:
df1['Prediction'].fillna(0, inplace=True)
# Display final DataFrame with predictions
print("Final DataFrame with Predictions:")
print(df1)

# Assuming ground_truth is defined outside the loop
# Calculate accuracy
ground_truth = df1.loc[test[sp].index, 'Label']
test_accuracy = accuracy_score(ground_truth, df1.loc[test[sp].index, 'Prediction'])
print("Test Accuracy:",test_accuracy)

Final DataFrame with Predictions:
     Label                               tokenized_Transcript  Prediction
0        1  represent people year system letter prisioner ...         0.0
1        1  reform criminal justice system matter law legi...         1.0
2        1  tell say accident say accident ian fine accide...         0.0
3        1  uh office get uh maybe close o clock remember ...         0.0
4        1                   yes yep stay uh yep prob yes yes         0.0
..     ...                                                ...         ...
116      0  believe evidence support domestic violence yes...         0.0
117      0  lot talk know seduce jodi frankly jodi seducti...         0.0
118      0  um person congregation involve activity like e...         0.0
119      0  statement ve candace conte molest candace cont...         0.0
120      0  oh word again think hear big inaudible like th...         0.0

[121 rows x 3 columns]
Test Accuracy: 0.5833333333333334


In [19]:
'''
empty space
















'''

'\nempty space\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'